In [18]:
import json
import pprint

class ApiObject:
    def __init__(self,**kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        keys=sorted(self.__dict__)
        items=("{}={!r}".format(k,self.__dict__[k]) for k in keys)
        return "{}({})".format(type(self).__name__,", ".join(items))

    def __eq__(self,other):
        return self.__dict__ == other.__dict__

    def create_from_response(response):
        return response.json(object_hook=lambda d: ApiObject(**d))

    def print(self):
        pprint.pprint(self.__dict__)

    def get(self,key):
        return self.__dict__[key]

    def __getattr__(self,name):
        return self.get(name)

In [2]:
import requests

HOST="100.124.54.98"
PORT=49986
NIFI_EP="http://%s:%d/nifi-api"%(HOST,PORT)

response=requests.get("%s/access"%(NIFI_EP))

pprint.pprint(response.text)
                      

'User authentication/authorization is only supported when running over HTTPS.'


In [3]:
response=requests.get("%s/controller/cluster"%(NIFI_EP))
pprint.pprint(response.json())


{'cluster': {'generated': '13:50:38 UTC',
             'nodes': [{'activeThreadCount': 0,
                        'address': 'd02c6cfe8e97',
                        'apiPort': 8080,
                        'events': [{'category': 'INFO',
                                    'message': 'Received first heartbeat from '
                                               'connecting node. Node '
                                               'connected.',
                                    'timestamp': '06/27/2023 18:38:44 UTC'},
                                   {'category': 'INFO',
                                    'message': 'Connection requested from '
                                               'existing node. Setting status '
                                               'to connecting.',
                                    'timestamp': '06/27/2023 18:38:41 UTC'}],
                        'heartbeat': '06/29/2023 13:50:36 UTC',
                        'nodeId': 'e216943a-122a-4469

In [4]:
EP="history"
query={"offset":0,"count":1}
response=requests.get("%s/flow/%s"%(NIFI_EP,EP),params=query)
if response.ok :
    pprint.pprint(response.json())
else:
    print(response.status_code,response.text)
    print(response.request.url)

{'history': {'actions': [{'action': {'actionDetails': {'name': 'hello',
                                                       'previousValue': 'world',
                                                       'value': 'steve'},
                                     'componentDetails': {'type': 'GenerateFlowFile'},
                                     'id': 10,
                                     'operation': 'Configure',
                                     'sourceId': '06ecaade-0189-1000-ffff-fffff5589410',
                                     'sourceName': 'fred',
                                     'sourceType': 'Processor',
                                     'timestamp': '06/29/2023 13:19:52 UTC',
                                     'userIdentity': 'anonymous'},
                          'canRead': True,
                          'id': 10,
                          'sourceId': '06ecaade-0189-1000-ffff-fffff5589410',
                          'timestamp': '06/29/2023 13:19:52 UTC

In [5]:
EP="search-results"
query={"q":"fred","a":""}
response=requests.get("%s/flow/%s"%(NIFI_EP,EP),params=query)
if response.ok :
    pprint.pprint(response.json())
else:
    raise Exception(response.status_code,response.text)

search_result=ApiObject.create_from_response(response)
processor=search_result.searchResultsDTO.processorResults[0]
pprint.pprint(processor.__dict__)

{'searchResultsDTO': {'connectionResults': [{'groupId': 'fe2564d6-0188-1000-8c26-92e2e002639c',
                                             'id': '073e7cd4-0189-1000-0000-00005e8f702f',
                                             'matches': ['Source name: fred'],
                                             'name': 'success',
                                             'parentGroup': {'id': 'fe2564d6-0188-1000-8c26-92e2e002639c',
                                                             'name': 'NiFi '
                                                                     'Flow'}}],
                      'controllerServiceNodeResults': [],
                      'funnelResults': [],
                      'inputPortResults': [],
                      'labelResults': [],
                      'outputPortResults': [],
                      'parameterContextResults': [],
                      'parameterProviderNodeResults': [],
                      'parameterResults': [],
             

In [23]:
EP="processors"
response=requests.get("%s/%s/%s"%(NIFI_EP,EP,processor.id))

if response.ok :
    print(response.status_code)
    p=ApiObject.create_from_response(response)
else:
    raise Exception(response.status_code,response.text)

#pprint.pprint(p.status.__dict__)

print(p.component.name,p.component.type)
props=p.component.config.properties
props.print()
print("Payload: ",props.get('generate-ff-custom-text'))
print("Mime Type: ",props.get('mime-type'))
print("Hello: ",props.hello)

SyntaxError: invalid syntax (1962553722.py, line 18)